# Outer Solar System IRNKGL-seq-BigFloat

## Loading packages and functions

In [6]:
using LinearAlgebra
using Plots
using JLD2
#FileIO
using Dates

In [2]:
PATH_SRC="../../src_seq/"

include(string(PATH_SRC,"IRKGL_SEQ.jl"))
using .IRKGL_SEQ    

In [7]:
setprecision(BigFloat, 256)

256

In [8]:
run=false

false



## Definition of the N-body problem

In Nbody.jl below, the following functions are defined: NbodyEnergy(u,Gm), NbodyODE!(du,u,Gm,t), and NbodyODE1!(du,u,Gm,t), where

\begin{equation*}
u = 
\left(\begin{matrix}
q_1  & v_1\\
\vdots & \vdots \\
q_N  & v_N
\end{matrix}
\right) \in \mathbb{R}^{2 \times 3\times N}, \quad
Gm = (G\,  m_1, \ldots,G\,  m_N) \in \mathbb{R}^N.
\end{equation*}

The energy, as a function of the positions $q_1,\ldots,q_N \in \mathbb{R}^3$ and the velocities $v_1,\ldots,v_N \in \mathbb{R}^3$ of the $N$ bodies is:
\begin{equation*}
\begin{split}
\mathrm{Energy}(q_1,\ldots,q_N,v_1,\ldots,v_N) = 
\frac12 \sum_{i=1}^{N} m_i \, \|v_i\|^2
- G \sum_{1\leq i < j \leq N} \frac{m_i m_j}{\|q_i-q_j\|}.
\end{split}
\end{equation*}


The ODE system of the N-body problem,
as a function of the positions $q_1,\ldots,q_N \in \mathbb{R}^3$ and the velocities $v_1,\ldots,v_N \in \mathbb{R}^3$ of the $N$ bodies is:
\begin{equation*}
\begin{split}
\frac{d}{dt} q_i &= v_i, \\
\frac{d}{dt} v_i &= G\, 
\sum_{j \neq i} \frac{m_j}{\|q_j-q_i\|^3}\,
(q_j-q_i).
\end{split}
\end{equation*}
This system of ODEs can be writen in compact form as
\begin{equation*}
\frac{d u}{dt} = f(t,u,Gm)
\end{equation*}




In [5]:
PATH_ODES="../../ODEProblems/"

include(string(PATH_ODES,"Initial15Body.jl"))
include(string(PATH_ODES,"Nbody.jl"))
#include(string(PATH_ODES,"Nbody2nd.jl"))

NbodyODE! (generic function with 2 methods)

##### <a href="#top">Back to the top</a>

## Initial value problem: 15-body problem (main asteroids)



In [6]:
u0, Gm, bodylist = Initial15Body(BigFloat)
q0=u0[:,:,1]
v0=u0[:,:,2]
dim=length(size(u0))

N = length(Gm)

show(bodylist)
E0=NbodyEnergy(u0,Gm)

["Sun" "Mercury" "Venus" "EMB" "Mars" "Jupiter" "Saturn" "Uranus" "Neptune" "Pluto" "Ceres" "Pallas" "Vesta" "Iris" "Bamberga"]

-9.831963632201247640647653709231152913256225158160386072963487864447472627831885e-12

### IRKNGL  integrazioa

In [7]:
now()

2022-12-17T16:48:00.141

In [8]:
t0 = BigFloat(0.)
dt = BigFloat(0.5)  
tF=  BigFloat(1e6)
tF = BigFloat(100*dt)
tF=BigFloat(15000.)


prob0 = ODEProblem(NbodyODE!, u0, (t0,tF), Gm)

nout=1000
m = Int64(ceil((tF-t0)/(nout*dt)))
println("dt = $dt, m=$m")

dt = 0.50, m=30


In [9]:
if run ==true
   alg=IRKNGL_Seq(s=8, initial_interp=0, m=m,myoutputs=true)
   sol0,iters0=solve(prob0,alg,dt=dt, adaptive=false)
   @save "./Data/sol_15body_QT1.jld2" sol0
   @save "./Data/prob_15body_QT1.jld2" prob0
else
   JLD2.@load "./Data/sol_15body_QT1.jld2" sol0
   JLD2.@load "./Data/prob_15body_QT1.jld2" prob0
end

LoadError: KeyError: key "sol0" not found

In [10]:
now()

2022-12-17T17:03:29.821

### Energy erroreak

In [11]:
eps(1e-25) 

1.1479437019748901e-41

In [12]:
yrange = (1e-45,1e-25)
year = 365.5

function energy_plot(sol; title="")
    energies = [NbodyEnergy(BigFloat.(u),Gm) for u in sol.u]
    E0 = energies[1]
    epsilon =eps(1e-25)                  #eps(1e-14)        #eps(1e-3)
    errors = (abs.(energies[2:end]/E0 .- 1)) .+ epsilon
    tt = sol.t[2:end]  #/year
    pl = plot(title=title,ylims=yrange,
         yscale=:log10, legend=false) #ylims=yrange, 
    plot!(tt,  errors)
    return pl
end

energy_plot (generic function with 1 method)

In [13]:
pl1 = energy_plot(sol0,  title="Energy error: 15-body problem")
plot(pl1)


LoadError: UndefVarError: sol0 not defined